In [9]:
import tensorflow as tf
import numpy as np
import cv2

In [10]:
img_height=480
img_width=640
#Network Parameters
n_input=img_height*img_width*3

batch_size=1

tf.reset_default_graph()
#tf graph input
x=tf.placeholder(tf.float32,[None,img_height,img_width,3])

In [11]:
def conv2d(img,w,b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def max_pool(img,k):
    return tf.nn.max_pool(img,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
def conv2d_transpose(img,w,b,outputShape):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d_transpose(img,w,output_shape=outputShape,strides=[1,1,1,1],padding='SAME'),b))

In [12]:
wc1=tf.Variable(tf.random_normal([5,5,3,16]),name="wc1")
bc1=tf.Variable(tf.random_normal([16]),name="bc1")

wc2=tf.Variable(tf.random_normal([5,5,16,64]),name="wc2")
bc2=tf.Variable(tf.random_normal([64]),name="bc2")

wc3=tf.Variable(tf.random_normal([5,5,64,64]),name="wc3")
bc3=tf.Variable(tf.random_normal([64]),name="bc3")

wc4=tf.Variable(tf.random_normal([5,5,16,64]),name="wc4")
bc4=tf.Variable(tf.random_normal([16]),name="bc4")

wc5=tf.Variable(tf.random_normal([5,5,16,16]),name="wc5")
bc5=tf.Variable(tf.random_normal([16]),name="bc5")

In [13]:
#Construct model
_X=tf.reshape(x,shape=[-1,img_height,img_width,3])
conv1_1=conv2d(_X,wc1,bc1)
conv1_2=max_pool(conv1_1,k=2)

conv2_1=conv2d(conv1_2,wc2,bc2)
conv2_2=max_pool(conv2_1,k=2)

conv3=conv2d(conv2_2,wc3,bc3)
conv3=max_pool(conv3,k=1)

add1=tf.add(conv2_2,conv3)
#add1=conv2+conv3

conv_t1=tf.image.resize_bilinear(add1,conv2_1.get_shape().as_list()[1:3])
tmpShape=conv1_2.get_shape().as_list()
tmpShape[0]=batch_size
conv_t1=conv2d_transpose(conv_t1,wc4,bc4,outputShape=tmpShape)

add2=tf.add(conv1_2,conv_t1)

conv_t2=tf.image.resize_bilinear(add2,conv1_1.get_shape().as_list()[1:3])
tmpShape=_X.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=16
conv_t2=conv2d_transpose(conv_t2,wc5,bc5,outputShape=tmpShape)

In [14]:
saver = tf.train.Saver()
init=tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [15]:
pairs=[[7,8],[8,9],[6,5],[5,4],[6,0],[7,0],[0,1],[1,2],[2,3],[10,11],[11,12],[15,14],[14,13],[12,1],[13,1]]
def draw_bones(img,joints):
    for line in pairs:
        a=(int(joints[line[0]][0]),int(joints_pos[line[0]][1]))
        b=(int(joints[line[1]][0]),int(joints_pos[line[1]][1]))
        cv2.line(img,a,b,(255,0,0),5)
    return img

In [16]:
with tf.Session(config=config) as sess:
    sess.run(init)
    saver.restore(sess,"model/model.ckpt")
    cap = cv2.VideoCapture(0)
    np_img=np.zeros([1,img_height,img_width,3])
    joints_pos=np.zeros([16,2])
    #tmpCount=0
    #while tmpCount<1:
    while True:
        ret, frame = cap.read()
        np_img[0]=np.array(frame)
        output=sess.run(conv_t2,feed_dict = {x:np_img})
        for i in range(0,16):
            tmp_heatmap=output[0,:,:,i]
            single_joint_location = np.unravel_index(tmp_heatmap.argmax(), tmp_heatmap.shape)
            joints_pos[i]=single_joint_location
        frame=draw_bones(frame,joints_pos)
        cv2.imshow('frame',frame)
        #tmpCount+=1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from model/model.ckpt
